<a href="https://colab.research.google.com/github/fowardelcac/Tp2_sem/blob/main/proccess_df_to_ddb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import sqlite3
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv("https://media.githubusercontent.com/media/fowardelcac/Tp2_sem/refs/heads/main/winemag-data-130k-v2.csv").drop('Unnamed: 0', axis = 1)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [15]:
def process(df: pd.DataFrame):
    # Función para imputar valores faltantes de forma aleatoria
    def random_imputer(df_filter: pd.DataFrame, col: str):
        # Seleccionar valores no nulos de la columna para la imputación
        non_null_values = df_filter[col].dropna().values

        # Aplicar un valor aleatorio de non_null_values donde hay NaN en la columna
        df_filter.loc[:, col] = df_filter[col].apply(
            lambda x: np.random.choice(non_null_values) if pd.isnull(x) else x
        )
        return df_filter

    # Crear una copia del DataFrame original
    df = df.copy()
    df.drop_duplicates(inplace=True)  # Eliminar duplicados
    df_filter = df.dropna(
        subset=["country", "designation"]
    )  # Filtrar filas con valores nulos en las columnas clave
    df_filter = random_imputer(
        df_filter, "price"
    )  # Imputar valores faltantes en la columna "price"
    return df_filter

In [16]:
df = process(df)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,80.0,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem


In [92]:
def build_country(df: pd.DataFrame):
  unique_countries = df['country'].dropna().unique()
  countries_df = pd.DataFrame(unique_countries, columns=['country'])
  countries_df['country_id'] = countries_df.index + 1
  return countries_df

def build_prov(df: pd.DataFrame, country_df: pd.DataFrame):
  unique_provinces = df[['province', 'country']].dropna().drop_duplicates()  # Obtener provincias con su país
  unique_provinces = unique_provinces.merge(country_df[['country_id', 'country']], on='country', how='left')  # Unir con la tabla de países
  provinces_df = unique_provinces[['province', 'country_id']]  # Seleccionar las columnas necesarias
  provinces_df['province_id'] = provinces_df.index + 1
  provinces_df.columns = ['province', 'country_id', 'province_id',]
  return provinces_df # Renombrar las columnas para mantener consistencia

def build_region(df: pd.DataFrame, province_df: pd.DataFrame):
  unique_region = df[['province', 'region_1']].dropna().drop_duplicates()
  unique_region = unique_region.merge(province_df[['province_id', 'province']], on='province', how='left')
  region_df = unique_region[['region_1', 'province_id']]
  region_df['region_id'] = region_df.index + 1
  region_df.columns = ['region', 'province_id', 'region_id']
  return region_df

def build_variety(df: pd.DataFrame):
  unique_varieties = df['variety'].dropna().unique()
  var_df = pd.DataFrame(unique_varieties, columns=['variety'])
  var_df['variety_id'] = var_df.index + 1
  return var_df.set_index('variety_id')

def build_winery(df: pd.DataFrame):
  unique_winery = df['winery'].dropna().unique()
  win_var = pd.DataFrame(unique_winery, columns=['winery'])
  win_var['winery_id'] = win_var.index + 1
  return win_var.set_index('winery_id')

def build_taster(df: pd.DataFrame):
  unique_taster = df['taster_name'].dropna().unique()
  taster_df = pd.DataFrame(unique_taster, columns=['taster_name'])

  taster_handles = df[['taster_name', 'taster_twitter_handle']].dropna().drop_duplicates()
  taster_df = taster_df.merge(taster_handles, on='taster_name', how='left')
  taster_df['taster_id'] = taster_df.index + 1
  return taster_df.set_index('taster_id')

In [93]:
country_table = build_country(df)
province_table = build_prov(df, country_table)
region_table = build_region(df, province_table)
variety_table = build_variety(df)
winery_table = build_winery(df)
taster_table = build_taster(df)

In [94]:
country_table.set_index("country_id", inplace = True)
province_table.set_index("province_id", inplace = True)
region_table.set_index("region_id", inplace = True)

In [96]:
region_table

,region,province_id
region_id,,
1,Etna,1
2,Lake Michigan Shore,3
3,Willamette Valley,4
4,Navarra,5
5,Vittoria,1
...,...,...
1133,Sonoma-Napa-Lake,8
1134,Sonoma County-Santa Barbara County,8
1135,Del Veneto,37


,winery,winery_id
0,Nicosia,1
1,Quinta dos Avidagos,2
2,St. Julian,3
3,Sweet Cheeks,4
4,Tandem,5


In [81]:
winery_table.set_index("winery_id")

,winery
winery_id,
1,Nicosia
2,Quinta dos Avidagos
3,St. Julian
4,Sweet Cheeks
5,Tandem
...,...
12534,Massanera
12535,Yacochuya
12536,Flagstone


,taster_name,taster_twitter_handle
taster_id,,
1,Kerin O’Keefe,@kerinokeefe
2,Roger Voss,@vossroger
3,Alexander Peartree,NaN
4,Paul Gregutt,@paulgwine
5,Michael Schachner,@wineschach
6,Anna Lee C. Iijima,NaN
7,Virginie Boone,@vboone
8,Matt Kettmann,@mattkettmann
9,Sean P. Sullivan,@wawinereport
